# Fetching Data

This file is pulling all the necessary informations from the github repositories and formatting it into the following JSON format:
- Name of Issue
- ID of the Issue
- Name of the Repo
- ID of the Repo
- Code Changes

In [38]:
# Parameters for the Notebook
# You can set the parameters to the ones you need before running the entire notebook

github_url: str = ""                # url of the repo, set only if you want a manual issue
manual_issue: bool = True           # 

# Set-Up of the Notebook

import pandas as pd
import numpy as np
import os
from github import Github
from dotenv import load_dotenv

load_dotenv()
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_OBJECT = Github(GITHUB_TOKEN)




### Creating the Database

The following code is only made to create a database of all the repos url I have to select from.

In [3]:
reactRepo = GITHUB_OBJECT.search_repositories(query=f'topic:react language:JavaScript language:TypeScript', sort="stars", order="desc")

In [75]:
for inx, repo in enumerate(reactRepo):
    issues = repo.get_issues(state="closed")
    for indx, issue in enumerate(issues):
        timeline = issue.get_timeline()
        for event in timeline:
            print(event.event)
        break
    break

committed
review_requested
labeled
commented
closed
mentioned
subscribed
head_ref_deleted
